Problem (Re)Statement:

* Shortness of breath (dyspnea) may be due to tuberculosis, lung cancer or bronchitis, or none of them, or more than one of them. 
* A recent visit to Asia increases the chances of tuberculosis.
* Smoking is known to be a risk factor for both lung cancer and bronchitis. 
* A positive chest X-ray suggests either lung cancer or tuberculosis, but cannot distinguish between them

Here is a data set to pull your model parameters from

In [35]:
from pandas import *
df = pandas.read_csv("asia.csv")

In [47]:
df.head()

,Smoker,LungCancer,VisitToAsia,Tuberculosis,XRay,Bronchitis,Dyspnea
0,1,1,0,0,1,1,1
1,0,0,0,0,1,1,1
2,0,0,0,0,0,1,1
3,0,0,0,0,0,1,1
4,1,0,0,0,0,1,0


<image src="asia.gif" size=200/>

<image src="asia.gif"/>

Begin by writing out your model.  For example here are names of some nodes, and the arcs that connect them.
<pre>
Asia                 -> Tuberculosis

Tuberculosis, Cancer -> Dyspnea

</pre>

-- Informally Write Your Model In This Cell -- 
It will determine the parameters you will need to get from the data set

Tuberculosis, Cancer, Bronchitis-> Dyspnea

Asia                 -> Tuberculosis

Smoker -> Cancer, Bronchitis

Lung Cancer, Tuberculosis -> X-ray

Now define your distribtions

In [38]:
from pomegranate import *

asia = df.VisitToAsia.value_counts()[1]/len(df.VisitToAsia)
asiadist =  DiscreteDistribution({0: (1-asia), 1: asia})

smoker = df.Smoker.value_counts()[1]/len(df.Smoker)
smokerdist = DiscreteDistribution({0: (1-smoker), 1: smoker})

tbcross = pandas.crosstab([df.VisitToAsia], [df.Tuberculosis], margins=True)
tbgivenasia = tbcross[1][1]/tbcross['All'][1]
tbgivennasia = tbcross[1][0]/tbcross['All'][0]

tbdist = ConditionalProbabilityTable(
        [[1, 1, tbgivenasia],
         [1, 0, 1-tbgivenasia],
         [0, 1, tbgivennasia],
         [0, 0, 1-tbgivennasia]], [asiadist])

cancercross = pandas.crosstab([df.Smoker], [df.LungCancer], margins=True)
cancergivensmoker = cancercross[1][1]/cancercross['All'][1]
cancergivennsmoker = cancercross[1][0]/cancercross['All'][0]

cancerdist = ConditionalProbabilityTable(
        [[1, 1, cancergivensmoker],
         [1, 0, 1-cancergivensmoker],
         [0, 1, cancergivennsmoker],
         [0, 0, 1-cancergivennsmoker]], [smokerdist])

bronchitiscross = pandas.crosstab([df.Smoker], [df.Bronchitis], margins=True)
bronchitisgivensmoker = bronchitiscross[1][1]/bronchitiscross['All'][1]
bronchitisgivennsmoker = bronchitiscross[1][0]/bronchitiscross['All'][0]

bronchitisdist = ConditionalProbabilityTable(
        [[1, 1, bronchitisgivensmoker],
         [1, 0, 1-bronchitisgivensmoker],
         [0, 1, bronchitisgivennsmoker],
         [0, 0, 1-bronchitisgivennsmoker]], [smokerdist])

dyspneacross = pandas.crosstab([df.Bronchitis, df.Dyspnea], [df.Tuberculosis, df.LungCancer], margins=True)

dyspneadist = ConditionalProbabilityTable(
        [[0, 0, 0, 0, dyspneacross[0][0][0][0]/(dyspneacross[0][0][0][0]+dyspneacross[0][0][0][1])],
         [0, 0, 0, 1, dyspneacross[0][0][0][1]/(dyspneacross[0][0][0][0]+dyspneacross[0][0][0][1])],
         [0, 0, 1, 0, dyspneacross[0][0][1][0]/(dyspneacross[0][0][1][0]+dyspneacross[0][0][1][1])],
         [0, 0, 1, 1, dyspneacross[0][0][1][1]/(dyspneacross[0][0][1][0]+dyspneacross[0][0][1][1])],
         [0, 1, 0, 0, dyspneacross[0][1][0][0]/(dyspneacross[0][1][0][0]+dyspneacross[0][1][0][1])],
         [0, 1, 0, 1, dyspneacross[0][1][0][1]/(dyspneacross[0][1][0][0]+dyspneacross[0][1][0][1])],
         [0, 1, 1, 0, dyspneacross[0][1][1][0]/(dyspneacross[0][1][1][0]+dyspneacross[0][1][1][1])],
         [0, 1, 1, 1, dyspneacross[0][1][1][1]/(dyspneacross[0][1][1][0]+dyspneacross[0][1][1][1])],
         [1, 0, 0, 0, dyspneacross[1][0][0][0]/(dyspneacross[1][0][0][0]+dyspneacross[1][0][0][1])],
         [1, 0, 0, 1, dyspneacross[1][0][0][1]/(dyspneacross[1][0][0][0]+dyspneacross[1][0][0][1])],
         [1, 0, 1, 0, dyspneacross[1][0][1][0]/(dyspneacross[1][0][1][0]+dyspneacross[1][0][1][1])],
         [1, 0, 1, 1, dyspneacross[1][0][1][1]/(dyspneacross[1][0][1][0]+dyspneacross[1][0][1][1])],
         [1, 1, 0, 0, dyspneacross[1][1][0][0]/(dyspneacross[1][1][0][0]+dyspneacross[1][1][0][1])],
         [1, 1, 0, 1, dyspneacross[1][1][0][1]/(dyspneacross[1][1][0][0]+dyspneacross[1][1][0][1])],
         [1, 1, 1, 0, dyspneacross[1][1][1][0]/(dyspneacross[1][1][1][0]+dyspneacross[1][1][1][1])],
         [1, 1, 1, 1, dyspneacross[1][1][1][1]/(dyspneacross[1][1][1][0]+dyspneacross[1][1][1][1])]
            ],[tbdist, cancerdist, bronchitisdist])


xraycross = pandas.crosstab([df.XRay], [df.Tuberculosis, df.LungCancer], margins=True)
xraydist = ConditionalProbabilityTable(
        [[0, 0, 0, xraycross[0][0][0]/(xraycross[0][0][0]+xraycross[0][0][1])],
         [0, 0, 1, xraycross[0][0][1]/(xraycross[0][0][0]+xraycross[0][0][1])],
         [0, 1, 0, xraycross[0][1][0]/(xraycross[0][1][0]+xraycross[0][1][1])],
         [0, 1, 1, xraycross[0][1][1]/(xraycross[0][1][0]+xraycross[0][1][1])],
         [1, 0, 0, xraycross[1][0][0]/(xraycross[1][0][0]+xraycross[1][0][1])],
         [1, 0, 1, xraycross[1][0][1]/(xraycross[1][0][0]+xraycross[1][0][1])],
         [1, 1, 0, xraycross[1][1][0]/(xraycross[1][1][0]+xraycross[1][1][1])],
         [1, 1, 1, xraycross[1][1][1]/(xraycross[1][1][0]+xraycross[1][1][1])]
            ], [tbdist, cancerdist])

Next define the nodes in your network

In [39]:
asia = Node(asiadist, name="Asia")
smoker = Node(smokerdist, name="Smoker")
tb = Node(tbdist, name="Tuberculosis")
cancer = Node(cancerdist, name="Lung Cancer")
bronchitis = Node(bronchitisdist, name="Bronchitis")
xray = Node(xraydist, name="X-Ray")
dyspnea = Node(dyspneadist, name="Dyspnea")

Define your model, adding states and edges

In [40]:
model = BayesianNetwork("Lung")
model.add_states(asia, smoker, tb, cancer, bronchitis, xray, dyspnea)
model.add_edge(asia, tb)
model.add_edge(smoker, cancer)
model.add_edge(smoker, bronchitis)
model.add_edge(tb, xray)
model.add_edge(tb, dyspnea)
model.add_edge(cancer, xray)
model.add_edge(cancer, dyspnea)
model.add_edge(bronchitis, dyspnea)
model.bake()    

------------------------------------------------

#### Questions

1.  What is the probability that an individual in the sampled population has either lung cancer or tuberculosis?

In [46]:
#The X-Ray probability being 1, or do I add TB and cancer and remove cross?
A = model.predict_proba({})[2].parameters[0][1]
B = model.predict_proba({})[3].parameters[0][1]
AgivenB = model.predict_proba({"Lung Cancer":1})[2].parameters[0][1]
AandB = AgivenB*B
print(A+B-AandB)

0.06535486000000125


2.  What is the probability that an individual in the sampled population will have a positive chest X-ray?  

In [42]:
model.predict_proba({})[5].parameters[0][1]

0.11052895971666488

3.  How much does a trip to Asia affect the likelihood of an individual having Dyspnea?

In [43]:
print((model.predict_proba({"Asia":1})[6].parameters[0][1])-(model.predict_proba({})[6].parameters[0][1]))

0.023797883148265253


It increases the odds by 2.37%

4.  Suppose you are a nonsmoker individual presenting with Dyspnea and you have never been to Asia.  In your panic you have a chest XRay done, which comes out negative.   What do you now know about the state of your health?  That is, do you believe you are more, less, or equally likely to have Tuberculosis, Lung Cancer, or Bronchitis?

In [44]:
model.predict_proba({"Smoker":0, "Dyspnea":1, "Asia":0, "X-Ray":0})

array([0, 0,
       {
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "0" :0.9994455574670935,
            "1" :0.0005544425329065244
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "0" :0.9993124030443784,
            "1" :0.0006875969556215444
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "0" :0.21993904950182494,
            "1" :0.780060950498175
        }
    ],
    "frozen" :false
},
       0, 1], dtype=object)

The chances of having tuberculosis or lung cancer is less than 0.1%. Although we can not be certain, there is a higher chance that you have bronchitis at 78%.